In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pandas import DataFrame
from pandas import concat

from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({'figure.figsize':(16,3), 'figure.dpi':100})

In [ ]:
# Leitura do arquivo Excel
series = pd.read_excel('spare-parts-sales.xlsx', header=0, 
                       names=['item', 'sales', 'cost', 'date'], index_col=None, parse_dates=True, squeeze=True)

# Filtra o arquivo para os anos de 2014 a 2016
series['date'] = pd.to_datetime(series['date'])
date_filter = (series['date'] >= '2014-01-01') & (series['date'] <= '2016-11-30')
series = series[date_filter]

# Filtra para o item '98550154' (óleo de motor)
sku = '98550154'
series = series[series.item == sku]

# Exclui a coluna item e cost
series.drop(series.columns[[0, 2]], axis=1, inplace=True)

# Transforma para vendas semanais
series = (series.groupby(pd.Grouper(key='date',freq='W')).sum()).reset_index()

In [ ]:
# transformar lista em formato de aprendizagem supervisionada
def series_to_supervised(data, n_in, n_out=1):
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg.values

In [ ]:
df = (pd.DataFrame(series_to_supervised(series['sales'], 1)))

In [ ]:
# Dividir em conjuntos de treinamento e teste
X = df.values

X = X.reshape((X.shape[0], X.shape[1], 1))

train_size = int(len(X) * 0.66)
train, test = X[0:train_size], X[train_size:]
train_X, train_y = train[:,0], train[:,1]
test_X, test_y = test[:,0], test[:,1]

In [ ]:
# Instanciando a rede neural
mlp = MLPRegressor(max_iter=500)

In [ ]:
# Dicionário de hiperparâmetros
parameter_space = {
    'hidden_layer_sizes': [(1,),(5,),(10,),(15,),(20,),(30,),(50,)],
    'activation': ["identity", "logistic", "tanh", "relu"],
    'solver': ["lbfgs", "sgd", "adam"],
    'alpha': [0.00005, 0.0005, 0.0001],
}

In [ ]:
# Grid Search
clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=5)
clf.fit(train_X, train_y.ravel())
print('Best parameters found:\n', clf.best_params_)

In [ ]:
#########################################################
# Executar o código a seguir após ajustar os parâmetros #
#########################################################

In [ ]:
mlp = MLPRegressor(activation='relu', alpha=0.0005, hidden_layer_sizes=(10,),
                   solver='lbfgs', max_iter=500)
mlp.fit(train_X, train_y.ravel())

In [ ]:
predictions=mlp.predict(test_X)

In [ ]:
# Avaliação do Modelo
MAPE = mean_absolute_percentage_error(test_y, predictions)
MAE = mean_absolute_error(test_y, predictions)
MSE = mean_squared_error(test_y, predictions)
R2 = r2_score(test_y,predictions)
accuracy = 100 - MAPE
print('Performance do Modelo')
print('----------------------------')
#print('Coeficiente de Determinação: {:0.2}.'.format(R2))
print('Acurácia = {:0.2f} %.'.format(accuracy))
print('MAPE = {:0.2f} %.'.format(MAPE))
print('MAE = {:0.2f} Unidades.'.format(MAE))
print('MSE = {:0.2f}.'.format(MSE))
print('RMSE = {:0.2f}.'.format(sqrt(MSE)))

In [ ]:
residuals = [test_y[i]-predictions[i] for i in range(len(predictions))]
residuals = DataFrame(residuals)
print('Descrição dos Resíduos')
print(residuals.describe())

In [ ]:
original=np.concatenate((train_X, test_X), axis=0)
plt.plot(original, label='Original')
x = range((len(train_X)), (len(original)))
plt.plot(x, predictions, label='Predicted')
plt.title('Vendas de Peças de Reposição de 2014 a 2016')
plt.xlabel('Meses')
plt.ylabel('Quantidade Vendas')
plt.legend(loc='best')
plt.show()